In [1]:
import keras
import os
import tensorflow as tf
from sklearn import preprocessing
from keras import layers, models, utils
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, LSTM
from sklearn.model_selection import train_test_split,cross_validate,cross_val_score, KFold
from keras.preprocessing import image
import joblib
import numpy as np
import cv2
import json
import sklearn.neural_network as MLPClassifier
import tf2onnx


In [123]:
class Model:

    def __init__(self):
        X, y = self.getDonneesTrain()

        self.model = Sequential()
        self.model.add(LSTM(64, input_shape=(X.shape[1], X.shape[2]), return_sequences=False))  # Couche LSTM
        self.model.add(Dense(32, activation='relu'))  # Couche dense cachée
        self.model.add(Dense(5, activation='softmax'))  # Couche de sortie avec activation softmax pour classification

        self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Ajustement des dimensions de y
        y = np.squeeze(y, axis=1)

        self.model.fit(X, y, epochs=100, batch_size=32)

    def getDonneesTrain(self):
        categories = ['1', '2', '3', '4', '5']
        labels = ["Depardieu", "Homer", "Merad", "Pesquet", "Tibo"]
        
        # Initialisation du LabelEncoder pour encoder les étiquettes
        le = preprocessing.LabelEncoder()
        le.fit(labels)
        
        # Transformation des labels en format catégoriel
        lab = le.transform(labels)
        lab = tf.keras.utils.to_categorical(lab)  # Conversion en one-hot encoding
        
        X = []
        y = []
        
        for i in range(len(categories)):
            nameFile = categories[i] + ".txt"
            path = os.path.join('.\\data\\train', nameFile)
            
            # Charger les données à partir du fichier JSON
            try:
                with open(path, 'r') as f:
                    tabData = json.load(f)  # Lecture du fichier JSON
            except FileNotFoundError:
                print(f"Erreur : fichier {path} non trouvé")
                continue
            except json.JSONDecodeError:
                print(f"Erreur de décodeur JSON pour le fichier {path}")
                continue
            
            print(len(tabData))
            print(tabData[0])
            # Ajout des données dans X et y
            for j in range(len(tabData)):
                X.append(tabData[j])
                y.append([lab[i]])
        
        # Conversion des listes en tableaux NumPy pour compatibilité avec Keras
        X = np.array(X, dtype=np.float32)
        y = np.array(y, dtype=np.float32)
        
        return X, y

In [124]:
model = Model()

20
[[0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [5, 0], [6, 0], [7, 0], [8, 0], [9, 0], [10, 0], [11, 0], [12, 0], [13, 0], [14, 0]]
10
[[0, 1], [1, 1], [2, 1], [3, 1], [4, 2], [5, 1], [6, 2], [7, 1], [8, 1], [9, 1], [10, 1], [11, 1], [12, 1], [13, 1], [14, 1]]
10
[[0, 3], [1, 2], [2, 2], [3, 3], [4, 2], [5, 3], [6, 2], [7, 3], [8, 3], [9, 2], [10, 3], [11, 2], [12, 2], [13, 3], [14, 2]]
11
[[0, 3], [1, 4], [2, 4], [3, 3], [4, 3], [5, 4], [6, 2], [7, 4], [8, 3], [9, 4], [10, 3], [11, 2], [12, 4], [13, 4], [14, 4]]
10
[[0, 3], [1, 4], [2, 4], [3, 3], [4, 4], [5, 4], [6, 2], [7, 4], [8, 2], [9, 3], [10, 4], [11, 2], [12, 4], [13, 3], [14, 2]]
Epoch 1/100
2/2 [==============================] - 2s 15ms/step - loss: 1.7826 - accuracy: 0.1311
Epoch 2/100
2/2 [==============================] - 0s 15ms/step - loss: 1.7112 - accuracy: 0.0492
Epoch 3/100
2/2 [==============================] - 0s 15ms/step - loss: 1.6637 - accuracy: 0.1803
Epoch 4/100
2/2 [==============================] - 0s 13ms/st

In [126]:
model.model.save('model.h5')
model = tf.keras.models.load_model('model.h5')

# Sauvegarder le modèle en tant que SavedModel (format nécessaire pour tf2onnx)
tf.saved_model.save(model, "saved_model")

c:\Users\Maxim\anaconda3\envs\ControlerVision\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [5]:
!python -m tf2onnx.convert --saved-model ".\\saved_model" --output saved_model.onnx

<frozen runpy>:128: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
2024-10-11 09:50:13,379 - WARNING - ***IMPORTANT*** Installed protobuf is not cpp accelerated. Conversion will be extremely slow. See https://github.com/onnx/tensorflow-onnx/issues/1557
2024-10-11 09:50:13,404 - WARNING - '--tag' not specified for saved_model. Using --tag serve
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\Maxim\anaconda3\envs\ControlerVision\Lib\site-packages\tf2onnx\convert.py", line 714, in <module>
    main()
  File "c:\Users\Maxim\anaconda3\envs\ControlerVision\Lib\site-packages\tf2onnx\convert.py", line 242, in main
    graph_def, inputs, outputs, initialized_tables, tensors_to_rename = tf_loader.from_saved_model(
                                              

In [ ]:
!powershell Compress-Archive -Path saved_model.onnx -DestinationPath saved_model.zip